In [ ]:
import numpy as np
import h5py
import os
# import pandas as pd
import yt
import matplotlib.pyplot as plt
from numba import jit
from typing import Generic, TypeVar, Callable, Tuple
from functools import reduce
import Pylians3
# h5py.run_tests()

In [ ]:
ds = yt.load('snapdir_000/snap_000.0.hdf5')
# ad = ds.all_data()

yt : [INFO     ] 2025-08-18 17:53:31,976 Calculating time from 2.507e-01 to be 6.810e+16 seconds
yt : [INFO     ] 2025-08-18 17:53:32,004 Parameters: current_time              = 6.8095920707497464e+16 s
yt : [INFO     ] 2025-08-18 17:53:32,005 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-08-18 17:53:32,005 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-08-18 17:53:32,005 Parameters: domain_right_edge         = [500. 500. 500.]
yt : [INFO     ] 2025-08-18 17:53:32,006 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-08-18 17:53:32,006 Parameters: current_redshift          = 2.9891842243418814
yt : [INFO     ] 2025-08-18 17:53:32,006 Parameters: omega_lambda              = 0.68685
yt : [INFO     ] 2025-08-18 17:53:32,006 Parameters: omega_matter              = 0.31315
yt : [INFO     ] 2025-08-18 17:53:32,007 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-08-18 17:53:32,007 Parameters: hubble_constant 

In [3]:
ds.r['PartType1', 'Coordinates']

yt : [INFO     ] 2025-08-18 17:53:34,052 Allocating for 1.074e+09 particles
Loading particle index: 100%|██████████| 4100/4100 [00:00<00:00, 12542.83it/s]


: 

In [4]:
print(ds.field_info["deposit", "PartType1_density"].get_source())

    def particle_density(field, data):
        pos = data[ptype, coord_name]
        pos.convert_to_units("code_length")
        mass = data[ptype, mass_name]
        mass.convert_to_units("code_mass")
        d = data.deposit(pos, [mass], method="sum")
        d = data.ds.arr(d, "code_mass")
        d /= data["index", "cell_volume"]
        return d



In [6]:
ds.field_list

[('PartType1', 'Coordinates'),
 ('PartType1', 'IntegerCoordinates'),
 ('PartType1', 'Masses'),
 ('PartType1', 'ModifiedGravityEffectiveMass'),
 ('PartType1', 'ModifiedGravityPhi'),
 ('PartType1', 'ParticleIDs'),
 ('PartType1', 'Velocities'),
 ('all', 'Coordinates'),
 ('all', 'IntegerCoordinates'),
 ('all', 'Masses'),
 ('all', 'ModifiedGravityEffectiveMass'),
 ('all', 'ModifiedGravityPhi'),
 ('all', 'ParticleIDs'),
 ('all', 'Velocities'),
 ('nbody', 'Coordinates'),
 ('nbody', 'IntegerCoordinates'),
 ('nbody', 'Masses'),
 ('nbody', 'ModifiedGravityEffectiveMass'),
 ('nbody', 'ModifiedGravityPhi'),
 ('nbody', 'ParticleIDs'),
 ('nbody', 'Velocities')]

In [7]:
ds.derived_field_list

[('PartType1', 'Coordinates'),
 ('PartType1', 'IntegerCoordinates'),
 ('PartType1', 'Masses'),
 ('PartType1', 'ModifiedGravityEffectiveMass'),
 ('PartType1', 'ModifiedGravityPhi'),
 ('PartType1', 'ParticleIDs'),
 ('PartType1', 'Velocities'),
 ('PartType1', 'particle_angular_momentum'),
 ('PartType1', 'particle_angular_momentum_magnitude'),
 ('PartType1', 'particle_angular_momentum_x'),
 ('PartType1', 'particle_angular_momentum_y'),
 ('PartType1', 'particle_angular_momentum_z'),
 ('PartType1', 'particle_index'),
 ('PartType1', 'particle_mass'),
 ('PartType1', 'particle_ones'),
 ('PartType1', 'particle_position'),
 ('PartType1', 'particle_position_cylindrical_radius'),
 ('PartType1', 'particle_position_cylindrical_theta'),
 ('PartType1', 'particle_position_cylindrical_z'),
 ('PartType1', 'particle_position_relative_x'),
 ('PartType1', 'particle_position_relative_y'),
 ('PartType1', 'particle_position_relative_z'),
 ('PartType1', 'particle_position_spherical_phi'),
 ('PartType1', 'particl

In [18]:
np.min(ds.r['PartType1', 'Coordinates']), np.max(ds.r['PartType1', 'Coordinates'])

(unyt_quantity(4.14438546e-07, 'code_length'),
 unyt_quantity(0.9999998, 'code_length'))

In [23]:
np.save('Coordinates.npy', ds.r['PartType1', 'Coordinates'])

In [24]:
np.save('Velocities.npy', ds.r['PartType1', 'Velocities'])

In [ ]:
print(ds.field_info["deposit", "PartType1_density"].get_source())

    def particle_density(field, data):
        pos = data[ptype, coord_name]
        pos.convert_to_units("code_length")
        mass = data[ptype, mass_name]
        mass.convert_to_units("code_mass")
        d = data.deposit(pos, [mass], method="sum")
        d = data.ds.arr(d, "code_mass")
        d /= data["index", "cell_volume"]
        return d



In [ ]:
print(ds.domain_width.in_units("kpc"))
print(ds.domain_width.in_units("au"))
print(ds.domain_width.in_units("mile"))

[186045.60843463 186045.60843463 186045.60843463] kpc
[3.83746614e+13 3.83746614e+13 3.83746614e+13] AU
[3.56714763e+21 3.56714763e+21 3.56714763e+21] mile


In [ ]:
print(ds.particle_types)
print(ds.particle_types_raw)
print(ds.particle_type_counts)

('PartType1', 'all', 'nbody')
('PartType1',)
{'PartType0': np.int64(0), 'PartType1': np.int64(1073741824), 'PartType2': np.int64(0), 'PartType3': np.int64(0), 'PartType4': np.int64(0), 'PartType5': np.int64(0)}


In [ ]:
1073741824**(1/3)

1023.9999999999997

In [ ]:
all_data = ds.all_data()

In [ ]:
all_data

YTRegion (snap_000): , center=[2.87038381e+26 2.87038381e+26 2.87038381e+26] cm, left_edge=[0. 0. 0.] cm, right_edge=[5.74076763e+26 5.74076763e+26 5.74076763e+26] cm